# T81-558: Applications of Deep Neural Networks
**Module 10: Time Series in Keras**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

* Part 10.1: Time Series Data Encoding for Deep Learning [[Video]](https://www.youtube.com/watch?v=dMUmHsktl04&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_1_timeseries.ipynb)
* Part 10.2: Programming LSTM with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=wY0dyFgNCgY&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_2_lstm.ipynb)
* **Part 10.3: Text Generation with Keras and TensorFlow** [[Video]](https://www.youtube.com/watch?v=6ORnRAz3gnA&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_3_text_generation.ipynb)
* Part 10.4: Image Captioning with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=NmoW_AYWkb4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_4_captioning.ipynb)
* Part 10.5: Temporal CNN in Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=i390g8acZwk&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_5_temporal_cnn.ipynb)

# Part 10.3: Text Generation with LSTM

Recurrent neural networks are also known for their ability to generate text.  This can allow the output of the neural network to be free-form text.  In this part we will see how an LSTM can be trained on a textual document, such as classic literature, and learn to output new text that appears to be of the same form as the training material.  If you train your LSTM on [Shakespeare](https://en.wikipedia.org/wiki/William_Shakespeare), then it will learn to crank out new prose that is similar to what Shakespeare had written. 

Don't get your hopes up.  Your not going to each your deep neural network to write the next [Pulitzer Prize for Fiction](https://en.wikipedia.org/wiki/Pulitzer_Prize_for_Fiction).  The prose generated by your neural network will be nonsensical.  However, it will usually be nearly grammatically and of a similar style as the source training documents. 

A neural network generating nonsensical text based on literature may not seem terribly useful at first glance.  However, the reason that this technology gets so much interest is that it forms the foundation for many more advanced technologies.  The fact that the LSTM will typically learn human grammar from the source document opens a wide range of possibilities. Similar technology can be used to complete sentences when a user is entering text.  Simply the ability to output free-form text becomes the foundation of many other technologies.  In the next part, we will make use of this technique to create a neural network that can write captions for images to describe what is going on in the image. 

### Additional Information

The following are some of the articles that I found useful putting this section together.

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [Keras LSTM Generation Example](https://keras.io/examples/lstm_text_generation/)

### Character-Level Text Generation

There are a number of different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, learning at the character level is the more interesting of the two.  The LSTM is learning construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  The automatic captioning that will be implemented in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed length array.  Not all of the sequence might be used, it is common to fill extra elements with zeros.  The text will be divided into sequences of this length and the neural network will be trained to predict what comes after this sequence.

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

For this simple example we will train the neural network on the classic children's book [Treasure Island](https://en.wikipedia.org/wiki/Treasure_Island).  We begin by loading this text into a Python string and displaying the first 1,000 characters.

In [2]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text
print(raw_text[0:1000])


ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Treasure Island

Author: Robert Louis Stevenson

Illustrator: Milo Winter

Release Date: January 12, 2009 [EBook #27780]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK TREASURE ISLAND ***




Produced by Juliet Sutherland, Stephen Blundell and the
Online Distributed Proofreading Team at http://www.pgdp.net









 THE ILLUSTRATED CHILDREN'S LIBRARY


         _Treasure Island_

       Robert Louis Stevenson

          _Illustrated by_
            Milo Winter


           [Illustration]


           GRAMERCY BOOKS
              NEW YORK




 Foreword copyright Â© 1986 by Random House V


We will extract all unique characters from the text and sort them.  This allows us to assign a unique ID to each character.  Because the characters are sorted, these IDs should remain the same.  If new characters were added to the original text, then the IDs would change.  We build up two dictionaries.  The first **char2idx** is used to convert a character into its ID.  The second **idx2char** converts an ID back into its character.

In [3]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text) 

In [4]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 397400
total chars: 60


We are now ready to build the actual sequences.  Just like previous neural networks there will be an $x$ and $y$.  However, for the LSTM, $x$ and $y$ will both be sequences.  The $x$ input will specify the sequences where $y$ are the expected output.  The following code generates all possible sequences.

In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 132454


In [6]:
sentences

['the project gutenberg ebook of treasure ',
 ' project gutenberg ebook of treasure isl',
 'oject gutenberg ebook of treasure island',
 'ct gutenberg ebook of treasure island, b',
 'gutenberg ebook of treasure island, by r',
 'enberg ebook of treasure island, by robe',
 'erg ebook of treasure island, by robert ',
 ' ebook of treasure island, by robert lou',
 'ook of treasure island, by robert louis ',
 ' of treasure island, by robert louis ste',
 ' treasure island, by robert louis steven',
 'easure island, by robert louis stevenson',
 'ure island, by robert louis stevenson\r\n\r',
 ' island, by robert louis stevenson\r\n\r\nth',
 'land, by robert louis stevenson\r\n\r\nthis ',
 'd, by robert louis stevenson\r\n\r\nthis ebo',
 'by robert louis stevenson\r\n\r\nthis ebook ',
 'robert louis stevenson\r\n\r\nthis ebook is ',
 'ert louis stevenson\r\n\r\nthis ebook is for',
 ' louis stevenson\r\n\r\nthis ebook is for th',
 'uis stevenson\r\n\r\nthis ebook is for the u',
 ' stevenson\r\n\r\n

In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
x.shape

(132454, 40, 60)

In [9]:
y.shape

(132454, 60)

The dummy variables for $y$ are shown below.

In [10]:
y[0:10]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, Fal

Next, we create the neural network.  The primary feature of this neural network is the LSTM layer.  This allows the sequences to be processed.  

In [11]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               96768     
_________________________________________________________________
dense (Dense)                (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


The LSTM will produce new text character by character.  We will need to sample the correct letter from the LSTM predictions each time.  The **sample** function accepts the following two parameters:

* **preds** - The output neurons.
* **temperature** - 1.0 is the most conservative, 0.0 is the most confident (willing to make spelling and other errors).

The sample function below is essentially performing a [softmax]() on the neural network predictions.  This causes each output neuron to become a probability of its particular letter.  

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


We are now ready to train.  It can take up to an hour to train this network, depending on how fast your computer is.  If you have a GPU available, please make sure to use it.

In [15]:
# Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# See https://github.com/tensorflow/tensorflow/issues/31308
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
132352/132454 [============================>.] - ETA: 0s - loss: 2.0195- ETA: 0s - loss****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "ction,
promotion and distribution of pr"
ction,
promotion and distribution of procus and the start of the store and the store and that at the captain and the store and the store and a proce and the captain and the store and the store and a procus and the captain the captain and the strow and the captain and the bore and the start and the boating the store and the store and the boat the boat and the sting and the store and the store and the store and the stold the store and th
----- temperature: 0.5
----- Generating with seed: "ction,
promotion and distribution of pr"
ction,
promotion and distribution of pronuted and dook and at the tated and streess and the sure and greeting sting and at i can to the dock, and sprong the capta

"doctor, "i began to get and the stores with and a moneyce. i had the crean and but the copinishand and silver of a greed his confirred in man was a sterned at the purered of the arms of the sea
shoulders, some till and the sure he had continue of the disting for a prec
----- temperature: 1.0
----- Generating with seed: "
    of declaring that this honest crea"

    of declaring that this honest creaticely coptain, re a shoulder
some man, "i thought and but he liefe, in the ewass
or and goatem, and
company."

"i repuies than caupher, but wend than word like what there nearely tile be-tilled to filler
and steptroprer portt of the island soon of the fien with himder tomery! the can eboors; plai saily muting us, and a lougher tuterys for where you? the see of the color
hole?"

he was tw
----- temperature: 1.2
----- Generating with seed: "
    of declaring that this honest crea"

    of declaring that this honest creak momenton in no, bug gladsgers, thome, and. ma, but duck of a but had , 

annywhing, and out nisll; in the doom, anchore, when
impar to loose not me boatey another. to think
hon."

alreas the super--is'll stand. they tell you verate for the bottmesh, then--there was
stommersly whan
mantainhern xpenly songle fime. with mind like viryve, aweriry. these escuplerol it's a mped. 
un!"
132454/132454 [==============================] - 338s 3ms/sample - loss: 1.3826
Epoch 8/60
132352/132454 [============================>.] - ETA: 0s - loss: 1.3635****************************************************************************
----- Generating text after Epoch: 7
----- temperature: 0.2
----- Generating with seed: "rew nearer in the
dark; i felt the haws"
rew nearer in the
dark; i felt the hawsing almost the captain side of the corner of the broken
the sides and stepped to the sandy and the doctor, and the rest of the shelp as and the more and behind me the cook and behind my mother behind my conter and the more to see the captain stood by the stood of the short was stand

cook said, and the same straighted on the stranger of the stockade, and the book and the sperry on the stranger of the
----- temperature: 0.5
----- Generating with seed: " lay as we had left him, on his back, wi"
 lay as we had left him, on his back, will re?"

"well," cried, "you have not the brisking to the sealar and the treasure--silver; for as the south terror of them they were in a third car of a shoulder that should been to the
black dog at the nergors of spock and put in the light best, and still carried the came of a shore. and a breathe ship had at
consting in the stockade, and seen musketes, and come at the same of
the captain, t
----- temperature: 1.0
----- Generating with seed: " lay as we had left him, on his back, wi"
 lay as we had left him, on his back, will us all this treowled hands are shap
possessed dises
a
hand in the captain smellen. theard thander up; too precett then thece who
help hakester in visibor at, that fwot hed was
nearer
for a mainsing seavaran genved


alrapy meding man,ly's arms. and and torserd. i'll do of a once good, i'll bad havily four to ti-docain.

"captany," then here!" cried there gentlecting talking the money rashle about of
iterher thunder. itt was
----- temperature: 1.2
----- Generating with seed: "king. "there's a power of men been kille"
king. "there's a power of men been killed, forgets, and
silbine bail took thunder ites, cimbe, cyme
busped up on the same of the cutlasses bag ar
where it, no perasi
portous all lading put the
passine on explondeming to given astocited nay you won't much bepwing us. as then in the have left me; myself cohade. but there oouth thalines incled over pinched.

hlay stee
goum,'s would old repuld out and saade? and,ly who; then old dan
132454/132454 [==============================] - 372s 3ms/sample - loss: 1.2958
Epoch 15/60
132352/132454 [============================>.] - ETA: 0s - loss: 1.2892****************************************************************************
----- Generating text

...

C:\Users\jheaton\AppData\Local\Continuum\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


132352/132454 [============================>.] - ETA: 0s - loss: 1.1720****************************************************************************
----- Generating text after Epoch: 47
----- temperature: 0.2
----- Generating with seed: "a suit of livery clothes, and such; that"
a suit of livery clothes, and such; that was a pipe of the stranger and seemed to the stranger of the blow of the stockade, and the ship was still the
sea that was a breath of the boat sailing the strotrore--but when i was a course, and at the same time
he had been what i seen the stockade, and i had seen the
sea that was a boy. and then he was a course, and then he was a palisade of the strong back and the blocked to the stranger of
----- temperature: 0.5
----- Generating with seed: "a suit of livery clothes, and such; that"
a suit of livery clothes, and such; that behind his head of the doctor was proped to got the
next moment with the nearest before a creachance in the stockade in
the best still project gute

"and there was ele't a sun was the last to and the same sort
of the cutlassed to be an ample and looking the party was nothing provent to
hands, not ready. i want to till it was a struck of
the boat, and we liked the cutlassed me.

i was a sea that's t
----- temperature: 1.0
----- Generating with seed: "come stony under foot, and the wood to c"
come stony under foot, and the wood to complete into the buriout, besinded by yourself.

wwo hit see. all's own to put to fever with. the treasure; mried
him, and thing us the idd anywared. i did

it     run ofiseate, my hearly treasure cutliblust of silver
had selsed at last thischiration, we goes
to sere out. laid, and saying my hand from a roust beside ebooked and tempera
beside the morrs, and seen me--to every in 'elest, str
----- temperature: 1.2
----- Generating with seed: "come stony under foot, and the wood to c"
come stony under foot, and the wood to cast
pated
-he first, "had got him is me weither's you you viiceburnecighting. mr,
dirk

intol; lay on the litc," near

captain knees she blau's fogrip had bly, you have so fwet.

"but us!" he cried, "whichogg briskionst eighuliny, sin, puck, and and mallow in a whold frum use had
them in the side, all since guir that with on'y from the nomest
ran in.

132454/132454 [==============================] - 405s 3ms/sample - loss: 1.1565
Epoch 55/60
132352/132454 [============================>.] - ETA: 0s - loss: 1.1555****************************************************************************
----- Generating text after Epoch: 54
----- temperature: 0.2
----- Generating with seed: ". so there's my
mind for you, my man, o"
. so there's my
mind for you, my man, or to be sure, and the sea-dage it was a sea add
to the stockade, and i say to that strong beach of the stockade, and the same time
you may comine, and i say to the same time the steep for the boats of the ship in the same time
he had the same time the stern and steep of the party was the ship before the boats of the stocka

bring and the boats, and the stockade, and the most singer of the shoulder of the corner of the ship's companions and the most side of the shores of the more and second of the shores of the storest and sure to the very blessing looked of the storest to the bottle of the shores of the boat
----- temperature: 0.5
----- Generating with seed: "m pirates came ashore, jim, what would y"
m pirates came ashore, jim, what would you
lives shot done had comed into the captain.

"the doctor sids"! and and what you can look the captain shot of the side of the doctor and stood by the other and sound of words of the red accoser and a little was flat water the boats, and the thing the
right and figure from his coance and disposition.

"the tailing of the neckon that i had till the
seamanlely with a between the trees, ther
----- temperature: 1.0
----- Generating with seed: "m pirates came ashore, jim, what would y"
m pirates came ashore, jim, what would you
cape, if i remeyded them," he were, and in hi